In [1]:
test_source_path = '/home/cadenc9020/summarization/dataset/fairseq_files/test-entswp-corrupted.source'
test_target_path = '/home/cadenc9020/summarization/dataset/fairseq_files/test-entswp-corrupted.target'
pred_target_path = 'test.hypo'

In [2]:
def read_summaries(file_path):
    lines = []
    with open(file_path) as f:
        for line in f:
            lines.append(line.strip())
    return lines

In [3]:
sources = read_summaries(test_source_path)

In [4]:
print(len(sources))

11327


In [5]:
targets = read_summaries(test_target_path)

In [6]:
print(len(targets))

11327


In [7]:
predicts = read_summaries(pred_target_path)

In [8]:
print(len(predicts))

11327


In [9]:
print(targets[0])
print(predicts[0])

Marseille prosecutor says "so far no videos were used in the crash investigation" despite media reports . Journalists at Bild and Paris Match are "very confident" the video clip is real, an editor says . Andreas Lubitz had informed his Lufthansa training school of an episode of severe depression, airline says .
Marseille prosecutor says "so far no videos were used in the crash investigation" despite media reports . Journalists at Bild and Paris Match are "very confident" the video clip is real, an editor says . Andreas Lubitz had informed his Lufthansa training school of an episode of severe depression, airline says .


In [14]:
match_num, print_count = 0, 0

for i, (s, t, p) in enumerate(zip(sources, targets, predicts)):
    if len(p) > len(t):
        p = p[: len(t)]
    s = s[: s.find('</s>')]
    
    t_tokens = t.split()
    p_tokens = p.split()

    if t_tokens == p_tokens:
        match_num += 1
    else:
        if print_count < 10:
            print('Target:')
            print(t)
            print('Sources:')
            print(s)
            print('Prediction:')
            print(p)
            print()
            print_count += 1

Target:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, Duke says . The noose, made of rope, was discovered on campus about 2 a.m.
Sources:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, North Carolina says . The noose, made of rope, was discovered on campus about 2 a.m. 
Prediction:
Student is no longer on Duke University campus and will face disciplinary review . School officials identified student during investigation and the person admitted to hanging the noose, Duke University says . The nooses, made of rope, was discovered on campus 

Target:
The Rev. Robert Schuller, 88, had been diagnosed with esophageal cancer in 2013 . His TV show, "Hour of Power," was enormously popular in the 1970s and 1980s .
Sources:

In [11]:
match_num / len(targets)

0.49280480268385274